### Raw 비트코인 데이터 전처리
- Object
    - 1분 단위로 전처리
    - 여러 파생변수(indicator) 생성

<br>

- python version: 3.8.18

#### 1) raw 데이터 불러와 bitcoin 데이터만 추출 후 저장

In [1]:
import dask.dataframe as dd
import gc # 가비지 컬렉터 모듈

In [2]:
# set csv file path
file_path = '../data/'

# set type of each columns
dtype={'code': 'object',
       'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# read csv file using Dask DataFrame
df = dd.read_csv(f'{file_path}ticker_data_202302230905.csv', dtype=dtype)

In [3]:
# df의 행 갯수: 79,238,904 -> 5분 소요
row_count = df['code'].count().compute()
print(row_count)

/opt/homebrew/lib/python3.8/site-packages/dask/dataframe/io/csv.py:193: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


79238904


In [4]:
print(df.columns)

Index(['type_websocket', 'datetime', 'code', 'opening_price', 'high_price',
       'low_price', 'trade_price', 'prev_closing_price', 'change',
       'change_price', 'signed_change_price', 'change_rate',
       'signed_change_rate', 'trade_volume', 'acc_trade_volume',
       'acc_trade_volume_24h', 'acc_trade_price', 'acc_trade_price_24h',
       'trade_date', 'trade_time', 'trade_timestamp', 'ask_bid',
       'acc_ask_volume', 'acc_bid_volume', 'highest_52_week_price',
       'highest_52_week_date', 'lowest_52_week_price', 'lowest_52_week_date',
       'market_state', 'is_trading_suspended', 'delisting_date',
       'market_warning', 'timestamp', 'stream_type', 'sys_datetime',
       'sequential_id', 'total_ask_size', 'total_bid_size', 'orderbook_ap_0',
       'orderbook_as_0', 'orderbook_bp_0', 'orderbook_bs_0', 'orderbook_ap_1',
       'orderbook_as_1', 'orderbook_bp_1', 'orderbook_bs_1', 'orderbook_ap_2',
       'orderbook_as_2', 'orderbook_bp_2', 'orderbook_bs_2', 'orderbook_ap_3'

In [4]:
df.head()

,type_websocket,datetime,code,opening_price,high_price,low_price,trade_price,prev_closing_price,change,change_price,...,orderbook_bp_12,orderbook_bs_12,orderbook_ap_13,orderbook_as_13,orderbook_bp_13,orderbook_bs_13,orderbook_ap_14,orderbook_as_14,orderbook_bp_14,orderbook_bs_14
0,orderbook,2023-01-21 20:33:43.000,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28519000.0,1.070351e+00,28583000.0,1.492000e-01,2.851800e+07,0.673184,28589000.0,1.662000e+00,2.851700e+07,0.054144
1,orderbook,2023-01-21 20:33:43.000,KRW-DOGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.5,1.109093e+06,121.0,3.595741e+06,9.940000e+01,284313.023536,122.0,4.639535e+06,9.930000e+01,429743.926727
2,orderbook,2023-01-21 20:33:43.000,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28519000.0,1.070351e+00,28583000.0,1.492000e-01,2.851800e+07,0.673184,28589000.0,1.662000e+00,2.851700e+07,0.054144
3,orderbook,2023-01-21 20:33:43.000,KRW-XRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,497.0,3.528720e+05,523.0,2.506845e+05,4.960000e+02,545004.723688,524.0,9.915758e+05,4.950000e+02,442778.375664
4,orderbook,2023-01-21 20:33:44.000,KRW-XRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,497.0,3.528720e+05,523.0,2.506845e+05,4.960000e+02,545004.723688,524.0,9.915758e+05,4.950000e+02,442778.375664


In [5]:
#-> 코드 작업에 5분 소요
unique_codes = df['code'].unique().compute()
print(unique_codes)

/opt/homebrew/lib/python3.8/site-packages/dask/dataframe/io/csv.py:193: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


0     KRW-BTC
1    KRW-DOGE
2     KRW-XRP
3     KRW-ETH
Name: code, dtype: object


In [6]:
#-> 코드 작업에 20분정도 소요(dtype다 지정해주면 더 빨라질지도?)
# 'code' 컬럼이 'KRW-BTC'에 해당하는 행만 필터링
bitcoin_df = df[df['code'] == "KRW-BTC"]

# 이전 DataFrame(df)의 참조를 제거하여 메모리 해제 요청 
del df
gc.collect()

# extract bitcoin data
#bitcoin_df.to_csv(f'{file_path}bitcoin_tick_data.csv', index=False)

478

In [7]:
# bitcoin_df의 행 갯수: 30,170,280 -> 5분 소요
row_count = bitcoin_df['code'].count().compute()
print(row_count)

/opt/homebrew/lib/python3.8/site-packages/dask/dataframe/io/csv.py:193: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


30170280


In [8]:
# 이전 DataFrame(df)의 참조를 제거하여 메모리 해제 요청 
del bitcoin_df
gc.collect()

0

#### 2) 추출된 bitcoin 데이터 불러와 duplicates 제거 및 30/60초 ticker 값 제거

In [ ]:
import dask.dataframe as dd
import gc # 가비지 컬렉터 모듈

In [11]:
# set csv file path
file_path = '../data/bitcoin_tick_data/'

# set type of each columns
dtype={'code': 'object',
       'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# read csv file using Dask DataFrame
df = dd.read_csv(f'{file_path}*.part', dtype=dtype)
#df = df.compute() # pandas df 형태로 반환. 메모리가 부족할 경우 추천하지 않음.

In [14]:
### Drop dupli
# 중복 제거 전 DataFrame의 행 길이 확인
print("df.shape 1 :", df.shape[0].compute())

# 첫 번째 중복 제거
df = df.drop_duplicates(subset=['type_websocket', 'timestamp', 'sys_datetime'], keep='first')
# 중복 제거 후 DataFrame의 행 길이 확인
print("df.shape 2 :", df.shape[0].compute())

# 두 번째 중복 제거
df = df.drop_duplicates(subset=['type_websocket', 'opening_price', 'high_price', 'low_price', 'trade_price', 'prev_closing_price', 'change', 'change_price', 'signed_change_price', 'change_rate', 'signed_change_rate', 'trade_volume', 'acc_trade_volume', 'acc_trade_price', 'trade_date', 'ask_bid', 'acc_ask_volume', 'acc_bid_volume', 'highest_52_week_price', 'highest_52_week_date', 'lowest_52_week_price', 'lowest_52_week_date', 'market_state', 'is_trading_suspended', 'total_ask_size', 'total_bid_size', 'orderbook_ap_0', 'orderbook_as_0', 'orderbook_bp_0', 'orderbook_bs_0', 'orderbook_ap_1', 'orderbook_as_1', 'orderbook_bp_1', 'orderbook_bs_1', 'orderbook_ap_2', 'orderbook_as_2', 'orderbook_bp_2', 'orderbook_bs_2', 'orderbook_ap_3', 'orderbook_as_3', 'orderbook_bp_3', 'orderbook_bs_3', 'orderbook_ap_4', 'orderbook_as_4', 'orderbook_bp_4', 'orderbook_bs_4', 'orderbook_ap_5', 'orderbook_as_5', 'orderbook_bp_5', 'orderbook_bs_5', 'orderbook_ap_6', 'orderbook_as_6', 'orderbook_bp_6', 'orderbook_bs_6', 'orderbook_ap_7', 'orderbook_as_7', 'orderbook_bp_7', 'orderbook_bs_7', 'orderbook_ap_8', 'orderbook_as_8', 'orderbook_bp_8', 'orderbook_bs_8', 'orderbook_ap_9', 'orderbook_as_9', 'orderbook_bp_9', 'orderbook_bs_9', 'orderbook_ap_10', 'orderbook_as_10', 'orderbook_bp_10', 'orderbook_bs_10', 'orderbook_ap_11', 'orderbook_as_11', 'orderbook_bp_11', 'orderbook_bs_11', 'orderbook_ap_12', 'orderbook_as_12', 'orderbook_bp_12', 'orderbook_bs_12', 'orderbook_ap_13', 'orderbook_as_13', 'orderbook_bp_13', 'orderbook_bs_13', 'orderbook_ap_14', 'orderbook_as_14', 'orderbook_bp_14', 'orderbook_bs_14'], keep='first')
# 최종 중복 제거 후 DataFrame의 행 길이 확인
print("df.shape 3 :", df.shape[0].compute())


/opt/homebrew/lib/python3.8/site-packages/dask/dataframe/io/csv.py:193: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


df.shape 1 : 30170280


/opt/homebrew/lib/python3.8/site-packages/dask/dataframe/io/csv.py:193: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


: 